In [1]:
import copy
import operator
import sre_parse
import sre_constants
from functools import reduce
from random import Random
from importlib import reload
from dataclasses import asdict
from typing import Tuple
from collections import defaultdict, Counter

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.markers import Path as MarkerPath
from matplotlib.transforms import Affine2D
from matplotlib.collections import PatchCollection
import seaborn as sns
from ipywidgets import Accordion, HTML, VBox, HBox, Output, Button, Layout, Dropdown, Tab, Widget, GridBox, Text
from ipywidgets import Label, GridspecLayout
from tqdm import tqdm

In [3]:
import constants
reload(constants)
import solver
reload(solver)

<module 'solver' from '/data/Development/regex_crossword/solver.py'>

In [4]:
strings = solver.build_strings()

In [5]:
with tqdm(strings, desc='init solutions') as progress:
    solution_sets = [solver.SparseSolutionSet.for_string(s) for s in progress]

init solutions: 100%|██████████| 39/39 [00:01<00:00, 38.73it/s]


In [6]:
s = reduce(lambda a, b: a.intersection(b), solution_sets)

In [7]:
len(s.cells)

127

In [8]:
cell_constraints = pd.Series([len(cc.constraint_dimensions) for cc in s.cells.values()])
cell_constraints.value_counts().sort_index()

1     21
2     19
3     22
4     27
5     14
6      3
7      1
8      8
9      3
10     3
12     1
15     1
16     1
19     1
20     1
25     1
dtype: int64

In [9]:
reduce(operator.mul, cell_constraints)

915277226268156478636335823267529417048457216000000000000000000000

In [10]:
cds = max(s.cells.values(), key=lambda cc: len(cc.constraint_dimensions)).constraint_dimensions
cds

((<LiteralConstraint [^X]>, <solver.DimensionCombination at 0x7f6a8ef08820>),
 (<CompoundConstraint [^X] : 5,5>,
  <solver.DimensionCombination at 0x7f6a8ef08880>),
 (<CompoundConstraint [^X] : 7,4>,
  <solver.DimensionCombination at 0x7f6a8f2d3070>),
 (<CompoundConstraint [^X] : 9,2>,
  <solver.DimensionCombination at 0x7f6a8ed80370>),
 (<CompoundConstraint [^X] : 11,0>,
  <solver.DimensionCombination at 0x7f6a8ed80460>),
 (<LiteralConstraint C>, <solver.DimensionCombination at 0x7f6a8ed80490>),
 (<CompoundConstraint C : 5,5>,
  <solver.DimensionCombination at 0x7f6a8ed808b0>),
 (<CompoundConstraint C : 7,4>,
  <solver.DimensionCombination at 0x7f6a8ed80850>),
 (<CompoundConstraint C : 9,2>,
  <solver.DimensionCombination at 0x7f6a8ed807f0>),
 (<CompoundConstraint C : 11,0>,
  <solver.DimensionCombination at 0x7f6a8ed807c0>),
 (<LiteralConstraint D>, <solver.DimensionCombination at 0x7f6a8ed80670>),
 (<CompoundConstraint D : 5,5>,
  <solver.DimensionCombination at 0x7f6a8ed800a0>),
 (

In [11]:
pd.Series(cds).map(lambda x: x[0])

0            [^X]
1      [^X] : 5,5
2      [^X] : 7,4
3      [^X] : 9,2
4     [^X] : 11,0
5               C
6         C : 5,5
7         C : 7,4
8         C : 9,2
9        C : 11,0
10              D
11        D : 5,5
12        D : 7,4
13        D : 9,2
14       D : 11,0
15              R
16        R : 5,5
17        R : 7,4
18        R : 9,2
19       R : 11,0
20              X
21        X : 5,5
22        X : 7,4
23        X : 9,2
24       X : 11,0
dtype: object

In [12]:
df = pd.DataFrame([{d:s for d,s in x.dimensions.items()} for _,x in cds],
                  index=[x for x,_ in cds])
df

,z3(21),z2(7),x4(1365),x3(683),x1(8),x6(1),x8(41),y6(66),x0(3),y1(45),...,x7(28),x9(89),z5(5),z6(3),z7(10),z8(7),z9(9),z10(55),z11(8),z12(44)
[^X],all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"[^X] : 5,5",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"[^X] : 7,4",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"[^X] : 9,2",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"[^X] : 11,0",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
C,all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all
"C : 5,5",all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all
"C : 7,4",all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all
"C : 9,2",all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all
"C : 11,0",all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all


In [13]:
df.sort_index()

,z3(21),z2(7),x4(1365),x3(683),x1(8),x6(1),x8(41),y6(66),x0(3),y1(45),...,x7(28),x9(89),z5(5),z6(3),z7(10),z8(7),z9(9),z10(55),z11(8),z12(44)
X,all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
C,all,all,all,all,all,all,all,all,all,all,...,+16,all,+1,all,all,all,all,all,all,all
R,all,all,all,all,all,all,all,all,all,all,...,+3,all,+1,all,all,all,all,all,all,all
D,all,all,all,all,all,all,all,all,all,all,...,+9,all,+1,all,all,all,all,all,all,all
[^X],all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"X : 7,4",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"X : 5,5",all,all,all,all,all,all,all,all,all,all,...,+13,all,+1,all,all,all,all,all,all,all
"R : 11,0",all,all,all,all,all,all,all,all,all,all,...,+3,all,+1,all,all,all,all,all,all,all
"R : 9,2",all,all,all,all,all,all,all,all,all,all,...,+3,all,+1,all,all,all,all,all,all,all
"R : 7,4",all,all,all,all,all,all,all,all,all,all,...,+3,all,+1,all,all,all,all,all,all,all


In [14]:
df.T.sort_index()

,[^X],"[^X] : 5,5","[^X] : 7,4","[^X] : 9,2","[^X] : 11,0",C,"C : 5,5","C : 7,4","C : 9,2","C : 11,0",...,R,"R : 5,5","R : 7,4","R : 9,2","R : 11,0",X,"X : 5,5","X : 7,4","X : 9,2","X : 11,0"
x0(3),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x1(8),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x2(5),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x3(683),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x4(1365),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x5(11),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x6(1),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x7(28),+13,+13,+13,+13,+13,+16,+16,+16,+16,+16,...,+3,+3,+3,+3,+3,+13,+13,+13,+13,+13
x8(41),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all
x9(89),all,all,all,all,all,all,all,all,all,all,...,all,all,all,all,all,all,all,all,all,all


In [18]:
df.iloc[0].iloc[0]

<DimensionSolutionAll>

In [19]:
ds_counts = df.apply(Counter).sort_index()
ds_counts

x0(3)                                 {all: 25}
x1(8)                                 {all: 25}
x2(5)                                 {all: 25}
x3(683)                               {all: 25}
x4(1365)                              {all: 25}
x5(11)                                {all: 25}
x6(1)                                 {all: 25}
x7(28)          {+13: 10, +16: 5, +9: 5, +3: 5}
x8(41)                                {all: 25}
x9(89)                                {all: 25}
x10(7)                                {all: 25}
x11(15)                               {all: 25}
x12(10)                               {all: 25}
y0(13)                                {all: 25}
y1(45)                                {all: 25}
y2(3)                                 {all: 25}
y3(12)                                {all: 25}
y4(41)                 {+16: 15, +33: 5, +8: 5}
y5(2)                                 {all: 25}
y6(66)                                {all: 25}
y7(27)                                {a

In [23]:
ds_counts_not_all = ds_counts[~ds_counts.map(lambda x: x.keys() == {solver.dimension_all})]
ds_counts_not_all

x7(28)        {+13: 10, +16: 5, +9: 5, +3: 5}
y4(41)               {+16: 15, +33: 5, +8: 5}
z5(5)     {+1: 5, +1: 5, +1: 5, +1: 5, +1: 5}
dtype: object

In [55]:
x = df[ds_counts_not_all.index].copy().sort_index()
x['combs'] = np.prod(x.applymap(lambda x: len(x.indexes)), axis=1)
x

,x7(28),y4(41),z5(5),combs
X,+13,+8,+1,104
C,+16,+33,+1,528
R,+3,+16,+1,48
D,+9,+16,+1,144
[^X],+13,+16,+1,208
"X : 7,4",+13,+8,+1,104
"X : 5,5",+13,+8,+1,104
"R : 11,0",+3,+16,+1,48
"R : 9,2",+3,+16,+1,48
"R : 7,4",+3,+16,+1,48


In [43]:
dim = ds_counts_not_all.index[0]
dim

<Dimension x7(28)>

In [44]:
df[dim].sort_index()

X              +13
C              +16
R               +3
D               +9
[^X]           +13
X : 7,4        +13
X : 5,5        +13
R : 11,0        +3
R : 9,2         +3
R : 7,4         +3
R : 5,5         +3
D : 11,0        +9
D : 9,2         +9
D : 7,4         +9
D : 5,5         +9
C : 11,0       +16
C : 9,2        +16
C : 7,4        +16
C : 5,5        +16
[^X] : 11,0    +13
[^X] : 9,2     +13
[^X] : 7,4     +13
[^X] : 5,5     +13
X : 9,2        +13
X : 11,0       +13
Name: x7(28), dtype: object

In [33]:
df[dim].sort_index().map(lambda x: sorted(x.indexes))

X              [0, 1, 2, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25]
C              [0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 19, 21, 22,...
R                                                   [20, 26, 27]
D                               [6, 7, 8, 9, 10, 11, 12, 13, 14]
[^X]           [0, 1, 2, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25]
X : 7,4        [0, 1, 2, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25]
X : 5,5        [0, 1, 2, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25]
R : 11,0                                            [20, 26, 27]
R : 9,2                                             [20, 26, 27]
R : 7,4                                             [20, 26, 27]
R : 5,5                                             [20, 26, 27]
D : 11,0                        [6, 7, 8, 9, 10, 11, 12, 13, 14]
D : 9,2                         [6, 7, 8, 9, 10, 11, 12, 13, 14]
D : 7,4                         [6, 7, 8, 9, 10, 11, 12, 13, 14]
D : 5,5                         [6, 7, 8, 9, 10, 11, 12, 13, 14]
C : 11,0       [0, 1, 2, 

In [36]:
pd.Series(list(reduce(lambda a,b: a | b, df[dim].sort_index().map(lambda x: x.indexes))))

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
20    20
21    21
22    22
23    23
24    24
25    25
26    26
27    27
dtype: int64

In [41]:
pd.Series(reduce(lambda a,b : a + b,
                 df[dim].sort_index().map(lambda x: list(x.indexes)))
).value_counts()

0     15
15    15
2     15
25    15
24    15
23    15
22    15
21    15
19    15
18    15
17    15
16    15
1     15
26     5
20     5
14     5
13     5
12     5
11     5
10     5
9      5
8      5
7      5
6      5
5      5
4      5
3      5
27     5
dtype: int64